### 聚类建表
#### 确定用户风格的关键字段：
1. 游戏币来源
    * 付费占比
    * 赢牌占比
    * 系统发放占比
    * 兑换占比
    
2. 玩牌场次
    * 高级场占比
    * 中级场占比
    * 低级场占比


对当日活跃用户进行统计，有需要的话，在这个基础上汇总统计出周表、月表即可（金条统一以1:100换算成游戏币）

#### 用户风格日表
* bpid
* mid
* 系统发放总量
* 系统发放占比
* 赢牌总量
* 赢牌占比
* 付费游戏币总量
* 付费占比
* 兑换总量
* 兑换占比
* 高级场玩牌时长
* 高级场玩牌局数
* 高级场时长占比
* 中级场玩牌时长
* 中级场玩牌局数
* 中级场时长占比
* 低级场玩牌时长
* 低级场玩牌局数
* 低级场时长占比
* 私人场玩牌时长
* 私人场玩牌局数
* 私人场时长占比
* 当日获取游戏币总量
* 当日游戏币流通总量
* dt

#### 建表步骤： 
* 为了通用性和保证正确，统一从流水表获取
* 表统一放在veda下面
* 每个游戏单独一张表（因为上报的货币来源id、游戏场次都不一样）
* 不同游戏由于游戏场次有所不同，并且上报的时候统一subname报的是id，先整理一份id对应的中文名的表(dcnew.gameparty_name_dim)
* 牌局从牌局日志stage.user_gameparty取
* 货币来源从游戏币流水取(stage.pb_gamecoins_stream)（analysis.game_coin_type_dim）
* 不同的act_id 对应到不同的货币来源，从bud里面整理出各个游戏对应的系统、付费、兑换的id列表(analysis.currencies_type_dim)
* 赢牌获取游戏币从牌局流水里面取

#### 自动推送方案
* 根据用户风格表，以及之前的聚类表，将用户进行分类，打上不同的标签；将标签建表，放在veda下面(veda.user_label_sirendoudizhu)
* 从dim.parquet_user_push 读取bpid、uid、token，从标签表(veda.user_label_sirendoudizhu)读取标签，从注册表取用户注册所在地区，通过bpid获取语言
* 推送文案：写在json文件里面，字段包括：游戏名称、平台id、bpid、语言、地区、类型、标题、内容
* 设置定时脚本，每日拉取表中日常维护的用户mid（检查推送回调表，抽取出返回成功的mid），调用推送接口，推送时间调用用户的登录时间，从文案库里面调取内容
* 将推送返回的mid、成功与否、失败原因、推送id、返回时间放入一张表

#### 推送效果方案
* 将推送失败的mid，与推送成功的mid进行对比
* 

In [1]:
#!/usr/local/python272/bin/python
# -*- coding: UTF-8 -*-

import os
import sys
import datetime
from sys import path
path.append('%s/' % os.getenv('SCS_PATH'))
from BaseStatModel import BaseStatModel
import service.sql_const as sql_const

class UserCoinSource(BaseStatModel):
    def create_tab(self):
        hql = """
        	create table if not exists veda.user_style_daily_sirendoudizhu
        	(
        		fbpid string ,
        		fuid bigint ,
        		sys_sum bigint comment '系统发放总量',
        		sys_percent double comment '系统发放占比（百分比）',
        		win_sum bigint comment '赢牌总量',
        		win_percent double comment '赢牌占比',
        		pay_sum bigint comment '付费总量',
        		pay_percent bigint comment '付费占比',
        		change_sum bigint comment '兑换总量',
        		change_percent double comment '兑换占比',
        		coin_total bigint comment '当日总共的游戏币获取数量',
        		primary_total_time bigint comment '初级场玩牌时长',
        		primary_total_count bigint comment '初级场玩牌局数',
        		primary_percent double comment '初级场玩牌时长占比',
        		practice_total_time bigint comment '练习场玩牌时长',
        		practice_total_count bigint comment '练习场玩牌局数',
        		practice_percent double comment '练习场玩牌时长占比',
        		native_total_time bigint comment '新手场玩牌时长',
        		native_total_count bigint comment '新手场玩牌局数',
        		native_percent double comment '新手场玩牌时长占比',
        		middle_total_time bigint comment '中级场玩牌时长',
        		middle_total_count bigint comment '中级场玩牌局数',
        		middle_percent double comment '中级场玩牌时长占比',
        		high_total_time bigint comment '高级场玩牌时长',
        		high_total_count bigint comment '高级场玩牌局数',
        		high_percent double comment '高级场玩牌时长占比',
        		master_total_time bigint comment '大师场玩牌时长',
        		master_total_count bigint comment '大师场玩牌局数',
        		master_percent double comment '大师场玩牌时长占比'
        	)
        	comment '四人斗地主用户风格表'
        	partitioned by (dt string comment '日期')
        	stored as parquet
        """
        res = self.sql_exe(hql)
        if res != 0:
            return res

    def stat(self):
	     	HQL_coin_source = """
				WITH b AS
				  (SELECT fbpid
				   FROM dim.bpid_map
				   WHERE fgamename = '四人斗地主'),
				     dim_coin AS
				  (SELECT ftype,
				          ftypename
				   FROM analysis.game_coin_type_dim
				   WHERE fdirection = 'IN'
				     AND fgamefsk = 1396896 ),
				     a1 AS
				  (SELECT DISTINCT a.fbpid,
				                   a.fuid,
				                   dim_coin.ftypename AS fbigtype,
				                   sum(if(a.act_num > 0, act_num, 0)) over (partition BY a.fbpid,a.fuid,dim_coin.ftypename,a.dt) AS coin_sum,
				                   sum(if(a.act_num > 0, act_num, 0)) over (partition BY a.fbpid,a.fuid,a.dt) AS coin_total_sum,
				                   a.dt
				   FROM stage.pb_gamecoins_stream a
				   inner JOIN b ON a.fbpid = b.fbpid
				   inner JOIN dim_coin ON a.act_id = dim_coin.ftype
				   WHERE a.act_type = 1 
				   and dt = '%(statdate)s'),
				     b1 AS
				  (SELECT fgameparty_name,
				          fdis_name
				   FROM dcnew.gameparty_name_dim
				   WHERE fgamefsk = 1396896 ),
				     b2 AS
				  (SELECT DISTINCT c.fbpid AS fbpid,
				                   c.fuid AS fuid,
				                   b1.fdis_name AS pname,
				                   sum(CASE
				                           WHEN unix_timestamp(c.fe_timer) - unix_timestamp(c.fs_timer) BETWEEN 1 AND 10800 THEN unix_timestamp(c.fe_timer) - unix_timestamp(c.fs_timer)
				                           ELSE 0
				                       END) over(partition BY c.fbpid,c.fuid) AS play_duration,
				                   sum(CASE
				                           WHEN unix_timestamp(c.fe_timer) - unix_timestamp(c.fs_timer) BETWEEN 1 AND 10800 THEN unix_timestamp(c.fe_timer) - unix_timestamp(c.fs_timer)
				                           ELSE 0
				                       END) over(partition BY c.fbpid,c.fuid,c.fsubname,c.dt) AS play_duration_subname,
				                   sum(c.fgamecoins) over(partition BY c.fbpid,c.fuid,c.dt) AS pname_coins_sum,
				                   sum(c.fgamecoins) over(partition BY c.fbpid,c.fuid,c.fsubname,c.dt) AS pname_coins,
				                   count(1) over(partition BY c.fbpid,c.fuid) AS pname_count_sum,
				                   count(1) over(partition BY c.fbpid,c.fuid,c.fsubname) AS pname_count,
				                   c.dt
				   FROM stage.user_gameparty c
				   inner JOIN b ON c.fbpid = b.fbpid
				   inner JOIN b1 ON c.fsubname = b1.fgameparty_name
				   WHERE c.fgamecoins >= 0 
				   and dt = '%(statdate)s'),
				     b3 AS
				  (SELECT fbpid,
				          fuid,
				          pname,
				          play_duration_subname,
				          pname_count,
				          cast(play_duration_subname*100/play_duration AS decimal(20,4)) AS pname_percent,
				          dt
				   FROM b2),
				     b4 AS
				  (SELECT b3.fbpid,
				          b3.fuid,
				          max(CASE
				                  WHEN b3.pname = '初级场' THEN b3.play_duration_subname
				                  ELSE 0
				              END) AS primary_total_time,
				          max(CASE
				                  WHEN b3.pname = '初级场' THEN b3.pname_count
				                  ELSE 0
				              END) AS primary_total_count,
				          max(CASE
				                  WHEN b3.pname = '初级场' THEN b3.pname_percent
				                  ELSE 0
				              END) AS primary_percent,
				          max(CASE
				                  WHEN b3.pname = '练习场' THEN b3.play_duration_subname
				                  ELSE 0
				              END) AS practice_total_time,
				          max(CASE
				                  WHEN b3.pname = '练习场' THEN b3.pname_count
				                  ELSE 0
				              END) AS practice_total_count,
				          max(CASE
				                  WHEN b3.pname = '练习场' THEN b3.pname_percent
				                  ELSE 0
				              END) AS practice_percent,
				          max(CASE
				                  WHEN b3.pname = '新手场' THEN b3.play_duration_subname
				                  ELSE 0
				              END) AS native_total_time,
				          max(CASE
				                  WHEN b3.pname = '新手场' THEN b3.pname_count
				                  ELSE 0
				              END) AS native_total_count,
				          max(CASE
				                  WHEN b3.pname = '新手场' THEN b3.pname_percent
				                  ELSE 0
				              END) AS native_percent,
				          max(CASE
				                  WHEN b3.pname = '中级场' THEN b3.play_duration_subname
				                  ELSE 0
				              END) AS middle_total_time,
				          max(CASE
				                  WHEN b3.pname = '中级场' THEN b3.pname_count
				                  ELSE 0
				              END) AS middle_total_count,
				          max(CASE
				                  WHEN b3.pname = '中级场' THEN b3.pname_percent
				                  ELSE 0
				              END) AS middle_percent,
				          max(CASE
				                  WHEN b3.pname = '高级场' THEN b3.play_duration_subname
				                  ELSE 0
				              END) AS high_total_time,
				          max(CASE
				                  WHEN b3.pname = '高级场' THEN b3.pname_count
				                  ELSE 0
				              END) AS high_total_count,
				          max(CASE
				                  WHEN b3.pname = '高级场' THEN b3.pname_percent
				                  ELSE 0
				              END) AS high_percent,
				          max(CASE
				                  WHEN b3.pname = '大师场' THEN b3.play_duration_subname
				                  ELSE 0
				              END) AS master_total_time,
				          max(CASE
				                  WHEN b3.pname = '大师场' THEN b3.pname_count
				                  ELSE 0
				              END) AS master_total_count,
				          max(CASE
				                  WHEN b3.pname = '大师场' THEN b3.pname_percent
				                  ELSE 0
				              END) AS master_percent,
				          dt
				   FROM b3
				   GROUP BY fbpid,
				            fuid,
				            dt),
				     c1 AS
				  (SELECT a1.fbpid,
				          a1.fuid,
				          a1.fbigtype,
				          a1.coin_sum,
				          b2.pname_coins_sum,
				          a1.coin_total_sum+b2.pname_coins_sum AS coin_sum_total,
				          a1.dt
				   FROM a1
				   JOIN b2 ON a1.fbpid = b2.fbpid
				   AND a1.fuid = b2.fuid
				   AND a1.dt = b2.dt),
				     c2 AS
				  (SELECT fbpid,
				          fuid,
				          fbigtype,
				          coin_sum,
				          pname_coins_sum,
				          cast(coin_sum*100/coin_sum_total AS decimal(20,4)) AS coin_percent_1,
				          cast(pname_coins_sum*100/coin_sum_total AS decimal(20,4)) AS coin_percent_2,
				          coin_sum_total,
				          dt
				   FROM c1),
				     c3 AS
				  (SELECT fbpid,
				          fuid,
				          max(CASE
				                  WHEN fbigtype ='系统' THEN coin_sum
				                  ELSE 0
				              END) AS sys_sum,
				          max(CASE
				                  WHEN fbigtype ='系统' THEN coin_percent_1
				                  ELSE 0
				              END) AS sys_percent,
				          max(CASE
				                  WHEN fbigtype ='支付' THEN coin_sum
				                  ELSE 0
				              END) AS pay_sum,
				          max(CASE
				                  WHEN fbigtype ='支付' THEN coin_percent_1
				                  ELSE 0
				              END) AS pay_percent,
				          max(CASE
				                  WHEN fbigtype ='兑换' THEN coin_sum
				                  ELSE 0
				              END) AS change_sum,
				          max(CASE
				                  WHEN fbigtype ='兑换' THEN coin_percent_1
				                  ELSE 0
				              END) AS change_percent,
				          pname_coins_sum AS win_sum,
				          coin_percent_2 AS win_percent,
				          coin_sum_total AS coin_total,
				          dt
				   FROM c2
				   GROUP BY fbpid,
				            fuid,
				            pname_coins_sum,
				            coin_percent_2,
				            coin_sum_total,
				            dt)
				insert overwrite table veda.user_style_daily_sirendoudizhu partition (dt = '%(statdate)s')
					SELECT c3.fbpid,
					c3.fuid,
					c3.sys_sum,
					c3.sys_percent,
					c3.pay_sum,
					c3.pay_percent,
					c3.change_sum,
					c3.change_percent,
					c3.win_sum,
					c3.win_percent,
					c3.coin_total,
					b4.primary_total_time,
					b4.primary_total_count,
					b4.primary_percent,
					b4.practice_total_time,
					b4.practice_total_count,
					b4.practice_percent,
					b4.native_total_time,
					b4.native_total_count,
					b4.native_percent,
					b4.middle_total_time,
					b4.middle_total_count,
					b4.middle_percent,
					b4.high_total_time,
					b4.high_total_count,
					b4.high_percent,
					b4.master_total_time,
					b4.master_total_count,
					b4.master_percent
				FROM c3
				LEFT JOIN b4 ON c3.fbpid = b4.fbpid
				AND c3.fuid = b4.fuid
				AND c3.dt = b4.dt
			
	     	"""
	     	res = self.sql_exe(HQL_coin_source)
	        if res != 0:
	            return res

        
        


# 实例化执行
a = UserCoinSource(sys.argv[1:])
a()

TabError: inconsistent use of tabs and spaces in indentation (<ipython-input-1-1f7c553774d2>, line 293)

### 接口代码

In [2]:
# 单条推送函数，传入bpid、token、推送标题、内容即可发送
def push_func(bpid,mid_token,title,content):
    import time
    import hashlib
    import json
    import urllib
    from urllib.parse import urlencode
    import requests
    from urllib.parse import quote 
    
    timeStamp = str(int(time.time()))
    req = 1
    token = 'c1fc8bf6bdc470b34e283451f2aaf47d'
    bpid = bpid
    title = title
    content = content
    usersInfo = json.dumps({"specify":mid_token})
    usersInfo2 = '%7B%22specify%22%3A+%22'+mid_token+'%22%7D'
    params = {
        'req':req,
        'timeStamp':timeStamp,
        'bpid':bpid,
        'title':quote(title),
        'content':quote(content),
        'usersInfo':usersInfo2}
    def sign(params,token):
        params['token'] = token
        pa_str = "&".join(['%s=%s' % (key, params[key]) for key in sorted(params.keys())])
        b = hashlib.md5(pa_str.encode('utf-8'))
        return b.hexdigest()

    sig = sign(params,token)
    
    url_content = {
            'req':req,
            'timeStamp':timeStamp,
            'sig':sig,
            'bpid':bpid,
            'title':title,
            'content':content,
            'usersInfo':usersInfo
    }
    
    url = 'http://pushd.boyaa.com/Api/send'
    return requests.post(url, data=url_content)

In [1]:
import numpy as np
import pandas as pd

push_config = pd.read_csv('./push_config.csv')
push_config

data = pd.read_csv('../1.csv')
bpid = data['fbpid'].values
token = data['ftoken'].values
label = data['label'].values




for i in range(0,len(data)):
    if label[i] == 0:
        push_func(bpid[i],token[i],title,'dear'+nickname[i]+content1)
    elif label[i] == 1:
        push_func(bpid[i],token[i],title,'dear'+nickname[i]+content2)
    else :
        push_func(bpid[i],token[i],title,'dear'+nickname[i]+content1)

NameError: name 'push_func' is not defined

In [14]:

push_config = pd.read_csv('./push_config.csv')
push_config


for i in range(0,len(push_config)):
    fbpid = push_config.iloc[i]['fbpid']
    fgamename = push_config.iloc[i]['fgamename']
    lang = push_config.iloc[i]['lang']
    location = push_config.iloc[i]['location']
    ftype = push_config.iloc[i]['type']
    label = push_config.iloc[i]['label']
    title = push_config.iloc[i]['title']
    content = push_config.iloc[i]['content']
    

<class 'str'>
<class 'str'>
